In [1]:
import pandas as pd
import time
import torch

## Загрузка данных

In [2]:
df = pd.read_csv('./data/data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [3]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [4]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [5]:
text[0]

['m',
 'a',
 'g',
 'g',
 'i',
 'e',
 ' ',
 'l',
 'o',
 'o',
 'k',
 ' ',
 'w',
 'h',
 'a',
 't',
 's',
 ' ',
 't',
 'h',
 'a',
 't']

## Делаем массив с данными то есть создаем наш словарь:

In [6]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [8]:
INDEX_TO_CHAR

['none',
 'r',
 'j',
 'f',
 'g',
 'z',
 's',
 'i',
 'y',
 'l',
 'x',
 'q',
 'h',
 ' ',
 'a',
 't',
 'e',
 'b',
 'p',
 'n',
 'd',
 'c',
 'u',
 'o',
 'w',
 'v',
 'm',
 'k']

In [9]:
CHAR_TO_INDEX

{'none': 0,
 'r': 1,
 'j': 2,
 'f': 3,
 'g': 4,
 'z': 5,
 's': 6,
 'i': 7,
 'y': 8,
 'l': 9,
 'x': 10,
 'q': 11,
 'h': 12,
 ' ': 13,
 'a': 14,
 't': 15,
 'e': 16,
 'b': 17,
 'p': 18,
 'n': 19,
 'd': 20,
 'c': 21,
 'u': 22,
 'o': 23,
 'w': 24,
 'v': 25,
 'm': 26,
 'k': 27}

In [10]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)

for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [11]:
X[0:10]

tensor([[26, 14,  4,  4,  7, 16, 13,  9, 23, 23, 27, 13, 24, 12, 14, 15,  6, 13,
         15, 12, 14, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 9, 16, 16,  0, 26, 22,  1, 13,  9, 16, 16,  0, 26, 22,  1,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 5, 16, 16,  0, 17, 23, 23, 13,  5, 16, 16,  0, 17, 23, 23,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7, 26, 13, 15,  1,  8,  7, 19,  4, 13, 15, 23, 13, 15, 16, 14, 21, 12,
         13, 26, 14,  4,  4,  7, 16, 13, 15, 12, 14, 15, 13, 19, 14, 15, 22,  1,
         16, 13, 20, 23, 16,  6, 19, 15, 13, 16, 19, 20, 13, 24],
        [ 7, 15,  6, 13,  9,  7, 27, 16, 13, 14, 19, 13, 23, 10, 13, 23, 19,  9,
       

## Смотрим на Embedding и RNN ячейку

In [13]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
t = embeddings(X[0:1])
t

tensor([[[-1.2979, -0.0067, -0.0343,  ..., -0.3603,  0.5358, -0.1214],
         [-0.5331, -1.5967, -0.8203,  ..., -0.8086, -0.8330,  0.8528],
         [ 0.0668,  0.0030,  0.3252,  ..., -0.1896,  0.7070, -0.4201],
         ...,
         [ 0.7193,  0.7376, -0.3651,  ...,  0.5934, -1.3055,  1.6610],
         [ 0.7193,  0.7376, -0.3651,  ...,  0.5934, -1.3055,  1.6610],
         [ 0.7193,  0.7376, -0.3651,  ...,  0.5934, -1.3055,  1.6610]]],
       grad_fn=<EmbeddingBackward0>)

In [14]:
t.shape, X[0:10].shape

(torch.Size([1, 50, 28]), torch.Size([10, 50]))

In [15]:
rnn = torch.nn.RNN(28, 128, batch_first=True)
o, s = rnn(t)
o.shape, s.shape

(torch.Size([1, 50, 128]), torch.Size([1, 1, 128]))

In [16]:
rnn = torch.nn.RNN(28, 128, batch_first=True)
o, s = rnn(t)
o.shape, s.shape

(torch.Size([1, 50, 128]), torch.Size([1, 1, 128]))

In [17]:
o, s2 = rnn(t, s)
o.shape, s2.shape

(torch.Size([1, 50, 128]), torch.Size([1, 1, 128]))

## Практика. Реализуйте код модели нейронной сети
3 слоя - embeding (28), скрытая ячейка (128), полносвязанный из состояния rnn в букву (28)

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        ## Здесь создать слои

    def forward(self, sentences, state=None):
        ## Здесь применить

In [ ]:
model = Network()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [ ]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

## Практика. Реализуйте код генерации следующей буквы на основе модели
Логика такая:
    - Сначала кормим в нее буквы из sentence (прогревая состояние)
    - Затем пока не получим none (0) берем самую вероятную букву и добавляем ее в sentence
    - Повторяем

In [ ]:
def generate_sentence():
    sentence = ['h', 'e', 'l', 'l', 'o']
    # Todo


In [ ]:
generate_sentence()

In [ ]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence()